In [3]:
from scipy.optimize import minimize
import numpy as np
from random import gauss
import scipy as sp

In [4]:
def center_image_fiducials(dimensions,fiducials):

    ''' Centers image fiducial points using image dimensions  '''

    for i in range(np.shape(fiducials)[0]):
        fiducials[i,0] = fiducials[i,0] - (dimensions[0] / 2)
        fiducials[i,1] = (dimensions[1] / 2) - fiducials[i,1]

    return fiducials

In [5]:
def colin(params, xyz_a):

	# Unwrap params
	kappa, phi, omega, xs, ys, zs, f = params

	omega = float(omega)
	phi = float(phi) + 0.5*np.pi
	kappa = float(kappa)
	xs = float(xs)
	ys = float(ys)
	zs = float(zs)
	f = float(f)

	# -- utils
	co = np.cos(omega)
	so = np.sin(omega)
	cp = np.cos(phi)
	sp = np.sin(phi)
	ck = np.cos(kappa)
	sk = np.sin(kappa)

	a1 =  cp*ck+sp*so*sk
	b1 =  cp*sk+sp*so*ck
	c1 =  sp*co
	a2 = -co*sk
	b2 =  co*ck
	c2 =  so
	a3 =  sp*ck+cp*so*sk
	b3 =  sp*sk-cp*so*ck
	c3 =  cp*co

	ynum  = a1*(xyz_a[:,0]-xs)+b1*(xyz_a[:,1]-ys)+c1*(xyz_a[:,2]-zs)
	xnum  = a2*(xyz_a[:,0]-xs)+b2*(xyz_a[:,1]-ys)+c2*(xyz_a[:,2]-zs)
	denom = a3*(xyz_a[:,0]-xs)+b3*(xyz_a[:,1]-ys)+c3*(xyz_a[:,2]-zs)

	xx = -f*xnum/denom
	yy = f*ynum/denom

	return np.vstack([xx,yy]).T

        '''	
	# Get number of fiducial points
	N = int(xyz_a.shape[0])

	# Initialize the result vector
	colin_xy = np.zeros((N,2))

	# Define c code that will evaluate the functions
	code = """

		double sinp = sinf(phi);
		double cosp = cosf(phi);
		double sino = sinf(omega);
		double coso = cosf(omega);
		double sink = sinf(kappa);
		double cosk = cosf(kappa);

		double a1 = cosp*cosk;
		double b1 = coso*sink + sino*sinp*cosk;
		double c1 = sino*sink - coso*sinp*cosk;
		double a2 = -1 * cosp*sink;
		double b2 = coso*cosk - sino*sinp*sink;
		double c2 = sino*cosk + coso*sinp*sink;
		double a3 = sinp;
		double b3 = -1*sino*cosp;
		double c3 = cosp*coso;

		int j;
		int k;
		double denom;

		for(int i = 0; i < N; i++)
		{
			j = i*2;
			k = i*3;
			denom = (a3*(xyz_a[k]-xs) + b3*(xyz_a[(k+1)]-ys) + c3*(xyz_a[(k+2)]-zs));

			colin_xy[j] = -1.0*f*(a1*(xyz_a[k]-xs) + b1*(xyz_a[(k+1)]-ys) + c1*(xyz_a[(k+2)]-zs))/denom;
			colin_xy[(j+1)] = -1.0*f*(a2*(xyz_a[k]-xs) + b2*(xyz_a[(k+1)]-ys) + c2*(xyz_a[(k+2)]-zs))/denom;
		}
		return_val = 1;
	"""

	# Use scipy.weave.inline to run the c code
	res = inline(code, ['colin_xy', 'omega', 'phi', 'kappa', 'xs', \
		'ys', 'zs', 'f', 'xyz_a', 'N'], headers = ['<math.h>'], \
		compiler = 'gcc')
	
	# Return the pixel (x,y) positions
	return colin_xy
	'''

In [37]:
def colinN(params, xyz_a):

	# Unwrap params
	kappa, phi, omega, xs, ys, zs, f = params

	omega = float(omega)*2*np.pi
	phi = (float(phi) + 0.5*np.pi)*np.pi
	kappa = float(kappa)*2*np.pi
	xs = (float(xs)*25000)+975000
	ys = (float(ys)*20000) + 192000
	zs = float(zs)*2000
	f = float(f)*3000

	# -- utils
	co = np.cos(omega)
	so = np.sin(omega)
	cp = np.cos(phi)
	sp = np.sin(phi)
	ck = np.cos(kappa)
	sk = np.sin(kappa)

	a1 =  cp*ck+sp*so*sk
	b1 =  cp*sk+sp*so*ck
	c1 =  sp*co
	a2 = -co*sk
	b2 =  co*ck
	c2 =  so
	a3 =  sp*ck+cp*so*sk
	b3 =  sp*sk-cp*so*ck
	c3 =  cp*co

	ynum  = a1*(xyz_a[:,0]-xs)+b1*(xyz_a[:,1]-ys)+c1*(xyz_a[:,2]-zs)
	xnum  = a2*(xyz_a[:,0]-xs)+b2*(xyz_a[:,1]-ys)+c2*(xyz_a[:,2]-zs)
	denom = a3*(xyz_a[:,0]-xs)+b3*(xyz_a[:,1]-ys)+c3*(xyz_a[:,2]-zs)

	xx = -f*xnum/denom
	yy = f*ynum/denom

	return np.vstack([xx,yy]).T

        '''	
	# Get number of fiducial points
	N = int(xyz_a.shape[0])

	# Initialize the result vector
	colin_xy = np.zeros((N,2))

	# Define c code that will evaluate the functions
	code = """

		double sinp = sinf(phi);
		double cosp = cosf(phi);
		double sino = sinf(omega);
		double coso = cosf(omega);
		double sink = sinf(kappa);
		double cosk = cosf(kappa);

		double a1 = cosp*cosk;
		double b1 = coso*sink + sino*sinp*cosk;
		double c1 = sino*sink - coso*sinp*cosk;
		double a2 = -1 * cosp*sink;
		double b2 = coso*cosk - sino*sinp*sink;
		double c2 = sino*cosk + coso*sinp*sink;
		double a3 = sinp;
		double b3 = -1*sino*cosp;
		double c3 = cosp*coso;

		int j;
		int k;
		double denom;

		for(int i = 0; i < N; i++)
		{
			j = i*2;
			k = i*3;
			denom = (a3*(xyz_a[k]-xs) + b3*(xyz_a[(k+1)]-ys) + c3*(xyz_a[(k+2)]-zs));

			colin_xy[j] = -1.0*f*(a1*(xyz_a[k]-xs) + b1*(xyz_a[(k+1)]-ys) + c1*(xyz_a[(k+2)]-zs))/denom;
			colin_xy[(j+1)] = -1.0*f*(a2*(xyz_a[k]-xs) + b2*(xyz_a[(k+1)]-ys) + c2*(xyz_a[(k+2)]-zs))/denom;
		}
		return_val = 1;
	"""

	# Use scipy.weave.inline to run the c code
	res = inline(code, ['colin_xy', 'omega', 'phi', 'kappa', 'xs', \
		'ys', 'zs', 'f', 'xyz_a', 'N'], headers = ['<math.h>'], \
		compiler = 'gcc')
	
	# Return the pixel (x,y) positions
	return colin_xy
	'''

In [6]:
def fullfunc(params,xyz_s,xy_t):

    ''' Find the sum of squares difference '''
    omega, phi, kappa, xs, ys, zs, f = params


#     if (omega<0.0) or (omega>=2.0*np.pi):
#         return 1e9+omega**4
#     elif (phi<-0.5*np.pi) or (phi>=0.5*np.pi):
#         return 1e9+phi**4
#     elif (kappa<0.0) or (kappa>=2.0*np.pi):
#         return 1e9+kappa**4
#     elif zs<0.0:
#         return 1e9+zs**4
#     elif f<0.0:
#         return 1e9+f**4

#     elif (np.abs(params[3] - 977119)>1000) or \
#             (np.abs(params[4] - 210445)>1000):
#         return 1e9 + xs**2

    colin_xy = 1.0*colin(params,xyz_s.astype(float))
    diff = ((colin_xy - xy_t)**2).sum()

    return diff

In [34]:
def call(params,xyz_s,xy_t):

    ''' Guess parameters near start and brute-force minimize '''
    start = params

    res = minimize(fullfunc, start,args=(xyz_s,xy_t), \
        method = 'L-BFGS-B', \
        options={'maxfev': 10000, 'maxiter': 10000})

    return res

In [17]:
lidar_fiducials = np.array([
                            [988328.741, 211962.778,1020.623838], #Empire state building
                            [980648.447, 199131.861,1380.707975], #WTC
                            [987656.616, 211766.233,493.89], # 1250 Broadway
                            [983564.98, 199358.775,591.406796], # Marshall courthouse
                            [987342.468, 212511.054,380.69], #  112 West 34th St
                            [988596.086, 211789.785,255.31], # 347 5th Ave
                            [988287.232, 213228.734,488.716947],  # 66 W 38th St
                            [984574.798, 204219.63, 303.263412], # 505 LaGuardia Place, second leftmost edge 
                            [987140.31, 209161.591, 324.317488], # Flatiron, left point on triange rooftop, under the fence
                            [986252.724, 205516.404, 365.468789]]) # 755 Broadway, leftmost corner on the roof
                            

In [18]:
xyz_s = lidar_fiducials

In [19]:
fiducials = np.array([
                        [856, 507],#Empire state building
                        [1004, 1681],# WTC
                        [1400, 1185], # 1250 Broadway
                        [1143, 1217], # Marshall courthouse
                        [1637, 1860], #  112 West 34th St
                        [1704, 211], # 347 5th Ave
                        [1811, 814], # 66 W 38th St
                        [1234, 1362], # 505 LaGuardia Place
                        [1355, 1037], # Flatiron
                        [1237, 1013]]) # 755 Broadway

dimensions = np.array([1918, 2560])
fiducials = center_image_fiducials(dimensions,fiducials)

In [13]:
xy_t = fiducials

In [36]:
guess = np.array([4.48603184, -0.3, 0, 988491, 214475, 800, 1000])
min_score = 100000000000000
num_iter = 100
params = guess
for i in range(0, num_iter):
    result = call(params,xyz_s,xy_t)
    print "params: ", result.x
    print "score: ", result.fun
    if (result.fun < min_score):# and (result.x[3] < 980491):
        min_score = result.fun
        params = result.x

C:\Users\victo\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: OptimizeWarning: Unknown solver options: maxfev


params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475000e+05   8.00000000e+02   1.00000000e+03]
score:  4945986.6424
params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475000e+05   8.00000000e+02   1.00000000e+03]
score:  4945986.6424
params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475000e+05   8.00000000e+02   1.00000000e+03]
score:  4945986.6424
params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475000e+05   8.00000000e+02   1.00000000e+03]
score:  4945986.6424
params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475000e+05   8.00000000e+02   1.00000000e+03]
score:  4945986.6424
params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475000e+05   8.00000000e+02   1.00000000e+03]
score:  4945986.6424
params:  [  4.48603184e+00  -3.00000000e-01   0.00000000e+00   9.88491000e+05
   2.14475